# Einops 'rearrange' Implementation from Scratch

This notebook implements a subset of the 'einops' library's 'rearrange' functionality using only Python and NumPy.

## Approach

The implementation follows a multi-step process:

1.  Parsing ('_parse_pattern', '_split_top_level', '_parse_composition'):
    *   The pattern string is split using a robust method ('_split_top_level') that respects parentheses, allowing for basic nesting like '(a (b c))'.
    *   Ellipsis ('...') can be placed anywhere in the pattern (but must appear consistently on both sides). Its position is recorded relative to the original specifications.
    *   Input and output specifications (like 'h', '(h w)', '((a b) c)') are processed to generate *flattened* lists of elementary identifiers (e.g., '((a b) c)' becomes ['a', 'b', 'c']) for permutation and internal logic. The original structured specifications are also retained to correctly map tensor dimensions during input analysis.
    *   Basic validation of identifiers and parenthesis balance occurs.

2.  Input Analysis:
    *   Tensor dimensions are matched against the input specifications (considering the original structure and ellipsis position) to identify prefix, ellipsis, and suffix dimensions.
    *   A dictionary 'axis_sizes' maps each *elementary* input axis identifier to its calculated size.
    *   *Splitting* operations (e.g., '(h w)' on input) are handled by analyzing the structured components, calculating sizes using the corresponding tensor dimension size and any provided 'axes_lengths'.

3.  Intermediate Reshape:
    *   The input tensor is reshaped (if necessary) into an "intermediate" form where dimensions correspond to the elementary axes identified in the input decomposition, preserving the relative order of ellipsis dimensions.
    *   (Note: Reshaping when ellipsis is in the middle of the pattern might be restricted if it requires complex dimension reordering that 'np.reshape' cannot handle directly).

4.  Output Analysis & Permutation:
    *   The output pattern's structure is analyzed to determine the shape elements of the final tensor, including calculating merged dimensions ('(...)' on output) and identifying axes for repetition (new axes specified in 'axes_lengths').
    *   A permutation map is calculated using the flattened elementary axis lists for input and output. This map specifies how to reorder the axes of the 'intermediate_tensor' (including the ellipsis dimensions) to match the desired output decomposition order.

5.  Transpose:
    *   'np.transpose' applies the calculated permutation to the (potentially reshaped) intermediate tensor.

6.  Repetition:
    *   For each genuinely *new* axis introduced in the output (present in 'axes_lengths' but not originating from an input axis), 'np.expand_dims' and 'np.repeat' are used to insert and tile this new dimension at the correct location in the permuted tensor. The insertion index is determined based on the final output decomposition order.
    *   Note on Semantics: This implementation uses tiling for new axes ('a b -> a b c'). It does *not* strictly enforce the 'einops' behavior of erroring if trying to "repeat" an *existing* axis that wasn't size 1 (e.g., 'a b c -> a d c' where b=2, d=5). It relies on NumPy's reshape/transpose behavior for such cases, which might error if sizes mismatch or succeed (effectively renaming) if they match.

7.  Final Reshape (Merging):
    *   The tensor (now transposed and with repeated axes added) is reshaped one last time, if necessary, to achieve the final target shape. This step handles *merging* axes specified by '(...)'' in the output pattern and ensures the ellipsis dimensions are in their correct final place.

8.  Error Handling: Includes checks for invalid patterns, shape mismatches, inconsistent/missing 'axes_lengths', parenthesis mismatch, invalid identifiers, unused input axes, and NumPy operation errors (reshape, transpose, repeat).

## Design Decisions & Improvements

*   Robust Parser: Uses a depth-counting splitter ('_split_top_level') and recursive composition parsing ('_parse_composition') to handle basic nesting like '(a (b c))'. Returns both original specs and flattened decomposition for different stages of the logic.
*   Flexible Ellipsis: Ellipsis ('...') is supported anywhere in the pattern (start, middle, end), provided it's consistent on both sides. The core logic handles shape calculations, permutation, and final reshaping around the ellipsis.
*   Intermediate Decomposition: Uses the strategy of decomposing specifications into elementary axes to determine sizes and permutations, simplifying the handling of combined operations like split-transpose-merge.
*   Repetition Behavior: Implements repetition by tiling new axes ('expand_dims' + 'repeat'). This is slightly more permissive than strict 'einops' for cases involving replacing existing non-1 dimensions but handles the common use case of adding/tiling new axes.

## Known Limitations

*   Complex Nesting/Parsing: While basic nesting '(a (b c))' is handled, extremely complex or ambiguous patterns might not parse correctly. The parser flattens nesting during decomposition, which might lose structural information needed for some hypothetical advanced patterns.
*   Anonymous '1' Mapping: Mapping input dimensions of size 1 specified via the literal '1' to output '1's relies on order and availability, which could be ambiguous in patterns with multiple '1's (e.g., '1 a 1 -> 1 1 a').
*   Performance: Relies on standard 'NumPy' operations ('reshape', 'transpose', 'repeat'), which may involve intermediate data copies and might not be as optimized as the backend-specific implementations in the actual 'einops' library, especially for very large tensors.
*   Strict Repeat Semantics: Does not strictly error when "replacing" a non-1 dimension with a sized dimension (e.g., 'a b c -> a d c' where b!=1), relying instead on 'NumPy's' behavior which might raise a 'ValueError' during reshape/transpose if sizes are incompatible.

## How to Run Tests

1.  Execute the Setup, Parser, and Core Implementation cells in order.
2.  Execute the Unit Tests cell. The 'unittest.main(...)' command will run all defined test classes ('TestEinopsRearrangeBase', 'TestEinopsRearrangeAdvanced', 'TestEinopsErrorHandling') and display the results.

In [ ]:
# CELL 1: Setup and Parsing Utilities
import numpy as np
import re
from typing import List, Tuple, Dict, Optional, Any, Sequence

class EinopsError(ValueError):
    """Custom exception for einops-related errors."""
    pass

def _split_top_level(s: str, separator=' ') -> List[str]:
    """Splits a string by a separator, respecting parentheses."""
    # (Your robust version from original setup code)
    parts = []
    current_part = ""
    depth = 0
    for char in s:
        if char == '(':
            depth += 1
            current_part += char
        elif char == ')':
            if depth == 0: raise ValueError(f"Mismatched parentheses: Extra closing parenthesis in '{s}'")
            depth -= 1
            current_part += char
        elif char == separator and depth == 0:
            if current_part: parts.append(current_part)
            current_part = ""
        else:
            current_part += char
    if depth != 0: raise ValueError(f"Mismatched parentheses: Unclosed parentheses in '{s}'")
    if current_part: parts.append(current_part)
    return parts

def validate_identifier(identifier: str):
    """Checks if an identifier is valid (alphanumeric or '1' or '')."""
    # (Your robust version from original setup code)
    if not re.fullmatch(r'[a-zA-Z0-9_]+|1', identifier):
        raise ValueError(f"Invalid identifier found: '{identifier}'")

def _parse_composition(composition: str) -> List[str]:
    """
    Parses a composition string like '(h w c)' or 'h' or '(a (b c))'
    into a flattened list of elementary identifiers. [REVISED VERSION]
    """
    composition = composition.strip()
    if not composition:
        return []
    # Use _split_top_level to handle top-level spaces correctly.
    # This will return ['h'] for 'h', and ['(h w)'] for '(h w)'.
    try:
        parts = _split_top_level(composition)
    except ValueError as e: # Catch errors like mismatched parens from splitter
        raise ValueError(f"Parsing error in '{composition}': {e}")

    # If the initial split gives more than one part, it means there were
    # top-level identifiers not enclosed in parentheses (e.g., "h w").
    if len(parts) > 1:
        raise ValueError(f"Invalid composition: multiple identifiers found without enclosing parentheses in '{composition}'. Found: {parts}")
    if not parts: # Should only happen if input was empty or only spaces
        return []

    # We now have exactly one logical part (it could be simple or parenthesized)
    single_part = parts[0]

    # Check if this part *is* a parenthesized expression
    if single_part.startswith('(') and single_part.endswith(')'):
        # Validate the balance of parentheses *within* this part before proceeding
        balance = 0
        valid_inner = True
        inner_content = single_part[1:-1] # Extract content within the outer parens
        for char in inner_content:
            if char == '(': balance += 1
            elif char == ')': balance -= 1
            if balance < 0: # Closing parenthesis before its opening one
                valid_inner = False; break
        # Final check: ensure balance is 0 at the end of inner content
        if balance != 0:
             valid_inner = False

        if not valid_inner:
            # This case means something like '((a)' or '(a))b(' was encountered
            raise ValueError(f"Mismatched parentheses within '{single_part}'")

        # If parentheses are balanced, parse the inner content
        inner_content_stripped = inner_content.strip()
        if not inner_content_stripped: # Handles '()' case
             return [] # An empty composition like '()' results in no elementary axes

        # Now, split the inner content by top-level spaces again
        # Example: if single_part was '((a b) c)', inner_content is '(a b) c'
        # _split_top_level on '(a b) c' should return ['(a b)', 'c']
        try:
            inner_components = _split_top_level(inner_content_stripped)
            # Recursively call _parse_composition on each of these components
            result = []
            for comp in inner_components:
                 result.extend(_parse_composition(comp)) # <<< RECURSION
            return result
        except ValueError as e:
             # Catch errors during the recursive parsing of inner components
             raise ValueError(f"Error parsing components within '{single_part}': {e}")

    else:
        # If the single_part wasn't enclosed in parentheses, it must be a single identifier.
        # Validate it directly.
        validate_identifier(single_part) # Corrected to use the actual function name
        return [single_part]

print("Parsing Utilities Defined")

Parsing Utilities Defined


In [ ]:
# CELL 2: Parser Definition (_parse_pattern)
# This cell uses the functions defined in the cell above

def _parse_pattern(pattern: str) -> Tuple[List[str], List[str], List[str], List[str], Optional[int], Optional[int]]:
    """
    Parses the einops pattern string, handling ellipsis and basic nesting.
    (Your version from the parser cell, unchanged)
    """
    # --- PASTE YOUR _parse_pattern CODE HERE ---
    if '->' not in pattern: raise EinopsError("Pattern must contain '->'")
    left_str, right_str = map(str.strip, pattern.split('->'))

    ellipsis_marker = "..."

    try:
        left_specs_orig = _split_top_level(left_str) # Uses _split_top_level from Cell 1
        right_specs_orig = _split_top_level(right_str)
    except ValueError as e: raise EinopsError(f"Failed parsing pattern structure: {e}")

    left_ellipsis_pos: Optional[int] = None; right_ellipsis_pos: Optional[int] = None
    left_specs_proc = list(left_specs_orig); right_specs_proc = list(right_specs_orig)

    if ellipsis_marker in left_specs_proc:
        if left_specs_proc.count(ellipsis_marker) > 1: raise EinopsError("Ellipsis (...) max once on left.")
        left_ellipsis_pos = left_specs_proc.index(ellipsis_marker)
        left_specs_proc.pop(left_ellipsis_pos)

    if ellipsis_marker in right_specs_proc:
         if right_specs_proc.count(ellipsis_marker) > 1: raise EinopsError("Ellipsis (...) max once on right.")
         right_ellipsis_pos = right_specs_proc.index(ellipsis_marker)
         right_specs_proc.pop(right_ellipsis_pos)

    if (left_ellipsis_pos is None) != (right_ellipsis_pos is None): raise EinopsError("Ellipsis (...) must be on both sides or neither.")

    left_elementary_axes: List[str] = []
    for spec in left_specs_proc:
        try:
            elementary = _parse_composition(spec) # Uses _parse_composition from Cell 1
            for identifier in elementary: validate_identifier(identifier) # Uses validate_identifier from Cell 1
            left_elementary_axes.extend(elementary)
        except ValueError as e: raise EinopsError(f"Failed parsing left component '{spec}': {e}")

    right_elementary_axes: List[str] = []
    for spec in right_specs_proc:
        try:
            elementary = _parse_composition(spec) # Uses _parse_composition from Cell 1
            for identifier in elementary: validate_identifier(identifier)
            right_elementary_axes.extend(elementary)
        except ValueError as e: raise EinopsError(f"Failed parsing right component '{spec}': {e}")

    counts = {}
    for ax in left_elementary_axes: counts[ax] = counts.get(ax, 0) + 1
    duplicates = [ax for ax, count in counts.items() if count > 1 and ax != '1']
    if duplicates: raise EinopsError(f"Duplicate axes on left: {duplicates}")

    return left_specs_orig, right_specs_orig, left_elementary_axes, right_elementary_axes, left_ellipsis_pos, right_ellipsis_pos
    # --- END OF _parse_pattern ---

print("Parser (_parse_pattern) Defined")

Parser (_parse_pattern) Defined


In [ ]:
# core.py (Version 6 - CLEANED, No Placeholders/Tests)

import numpy as np
import re
from typing import List, Tuple, Dict, Optional, Any, Sequence

# Assume parsing functions (_parse_pattern, _parse_composition, etc.) and
# EinopsError class are defined in preceding cells in the Colab environment.

# Define EinopsError here if it wasn't defined in the setup cell
# (Alternatively, ensure it's defined in Cell 1)
class EinopsError(ValueError):
    """Custom exception for einops-related errors."""
    pass

# NOTE: No placeholder definitions for parsing functions here.
# The rearrange function relies on them being defined in the Colab runtime.

def rearrange(tensor: np.ndarray, pattern: str, **axes_lengths: int) -> np.ndarray:
    """
    Rearranges dimensions of a tensor according to the einops pattern.
    (Version 6 - Relies on externally defined parsers)
    """
    # PARSING STEP: Assumes _parse_pattern is defined correctly in the environment
    try:
        # These functions (_parse_pattern, _parse_composition) MUST be
        # the robust versions defined and executed in previous Colab cells.
        input_specs_orig, output_specs_orig, \
        input_decomp_flat, output_decomp_flat, \
        left_ellipsis_pos, right_ellipsis_pos = _parse_pattern(pattern)
    except (ValueError, EinopsError) as e:
         raise EinopsError(f"Invalid pattern: '{pattern}'. Reason: {e}")

    # --- Input Shape Analysis ---
    has_ellipsis = left_ellipsis_pos is not None; tensor_ndim = tensor.ndim
    input_specs_proc = [spec for spec in input_specs_orig if spec != "..."]
    input_ndim_pattern_specs = len(input_specs_proc)
    ellipsis_shape: Tuple[int, ...] = (); num_ellipsis_dims = 0
    input_shape_processed: Tuple[int, ...] = ()
    if has_ellipsis:
        if tensor_ndim < input_ndim_pattern_specs: raise EinopsError(f"Tensor ndim {tensor_ndim} < pattern non-ellipsis {input_ndim_pattern_specs}")
        num_ellipsis_dims = tensor_ndim - input_ndim_pattern_specs
        num_prefix_specs = left_ellipsis_pos
        prefix_spec_shape = tensor.shape[:num_prefix_specs]; ellipsis_shape = tensor.shape[num_prefix_specs : num_prefix_specs + num_ellipsis_dims]
        suffix_spec_shape = tensor.shape[num_prefix_specs + num_ellipsis_dims:]
        if len(prefix_spec_shape) + len(suffix_spec_shape) != input_ndim_pattern_specs: raise EinopsError(f"Shape {tensor.shape} doesn't match non-ellipsis specs {input_ndim_pattern_specs}")
        input_shape_processed = prefix_spec_shape + suffix_spec_shape
    else:
        if tensor_ndim != input_ndim_pattern_specs: raise EinopsError(f"Tensor ndim {tensor_ndim} != pattern specs {input_ndim_pattern_specs}")
        input_shape_processed = tensor.shape

    # --- Axis Size Calculation ---
    # Assumes _parse_composition is defined correctly in the environment
    axis_sizes: Dict[str, int] = {}; current_dim_index = 0; used_axes_lengths = set(); decomp_flat_idx_ptr = 0; input_axis_names_set = set()
    for spec_idx, spec in enumerate(input_specs_proc):
        if current_dim_index >= len(input_shape_processed): raise EinopsError(f"Internal shape index error")
        dim_size = input_shape_processed[current_dim_index]
        try:
            components = _parse_composition(spec) # Uses function defined externally
            num_components = len(components); current_spec_components_flat = input_decomp_flat[decomp_flat_idx_ptr : decomp_flat_idx_ptr + num_components]
            input_axis_names_set.update(current_spec_components_flat)
        except Exception as e: raise EinopsError(f"Failed parsing input spec '{spec}': {e}")
        # (Rest of axis size calculation logic - identical to previous correct versions)
        if not components and spec == '1':
             if dim_size != 1: raise EinopsError(f"Spec '{spec}' requires 1, got {dim_size}.")
             axis_sizes['1'] = 1; input_axis_names_set.add('1')
        elif len(components) == 1:
            identifier = components[0]; flat_axis_name = current_spec_components_flat[0]
            if identifier == '1':
                if dim_size != 1: raise EinopsError(f"Spec '{spec}' requires 1, got {dim_size}.")
                axis_sizes['1'] = 1; input_axis_names_set.add('1')
            else:
                if flat_axis_name in axis_sizes and axis_sizes[flat_axis_name] != dim_size: raise EinopsError(f"Axis '{flat_axis_name}' conflicting sizes.")
                axis_sizes[flat_axis_name] = dim_size
        elif len(components) > 1:
            unknown_components: List[str] = []; known_size_product = 1; current_elementary_axis_sizes: Dict[str, int] = {}; axes_provided_for_split = set()
            for comp_idx, comp in enumerate(components):
                flat_axis_name = current_spec_components_flat[comp_idx]
                if comp == '1': current_elementary_axis_sizes[flat_axis_name] = 1; known_size_product *= 1; input_axis_names_set.add('1'); continue
                size_found = False
                if comp in axes_lengths:
                    size = axes_lengths[comp];
                    if not isinstance(size, int) or size <= 0: raise EinopsError(f"Size for '{comp}' invalid.")
                    current_elementary_axis_sizes[flat_axis_name] = size; known_size_product *= size; used_axes_lengths.add(comp); axes_provided_for_split.add(comp); size_found = True
                elif flat_axis_name in axis_sizes: size = axis_sizes[flat_axis_name]; current_elementary_axis_sizes[flat_axis_name] = size; known_size_product *= size; size_found = True
                if not size_found: unknown_components.append(flat_axis_name)
            if len(unknown_components) > 1: raise EinopsError(f"Cannot infer sizes for {unknown_components} in '{spec}'.")
            elif len(unknown_components) == 1:
                unknown_flat_name = unknown_components[0]
                if known_size_product <= 0: raise EinopsError(f"Internal product <= 0 for '{spec}'.")
                if dim_size % known_size_product != 0: raise EinopsError(f"Dim {dim_size} not divisible by {known_size_product} for '{spec}'.")
                calculated_size = dim_size // known_size_product
                if calculated_size <= 0: raise EinopsError(f"Inferred size <= 0 for '{unknown_flat_name}'.")
                current_elementary_axis_sizes[unknown_flat_name] = calculated_size
            elif len(unknown_components) == 0:
                 final_product = np.prod(list(current_elementary_axis_sizes.values()), dtype=np.int64)
                 if final_product != dim_size: raise EinopsError(f"Product {final_product} != dim {dim_size} for '{spec}'.")
            for flat_name, size in current_elementary_axis_sizes.items():
                 if flat_name in axis_sizes and axis_sizes[flat_name] != size: raise EinopsError(f"Axis '{flat_name}' conflicting sizes.")
                 axis_sizes[flat_name] = size
        elif spec == '()':
             if dim_size != 1: raise EinopsError(f"Spec '()' requires 1, got {dim_size}.")
        else: raise EinopsError(f"Invalid spec component: '{spec}'.")
        decomp_flat_idx_ptr += num_components; current_dim_index += 1

    # --- Intermediate Reshape ---
    # Assumes _parse_composition is defined correctly in the environment
    intermediate_shape_elements = []
    for ax in input_decomp_flat:
        size = axis_sizes.get(ax)
        if size is None:
            if ax == '1': size = 1
            else: raise EinopsError(f"Internal error: Size for axis '{ax}' not found.")
        intermediate_shape_elements.append(size)
    core_target_shape = tuple(intermediate_shape_elements); reshaped_input = tensor; target_intermediate_full_shape: Tuple[int, ...]
    if has_ellipsis:
        num_prefix_elems = sum(len(_parse_composition(input_specs_proc[i])) for i in range(left_ellipsis_pos)) # Uses function defined externally
        target_intermediate_full_shape = core_target_shape[:num_prefix_elems] + ellipsis_shape + core_target_shape[num_prefix_elems:]
    else: target_intermediate_full_shape = core_target_shape
    if tensor.shape != target_intermediate_full_shape:
         if np.prod(tensor.shape) != np.prod(target_intermediate_full_shape): raise EinopsError(f"Intermediate reshape count mismatch.")
         try: reshaped_input = np.reshape(tensor, target_intermediate_full_shape)
         except Exception as e: raise EinopsError(f"Intermediate reshape failed: {e}")

    # --- Output Analysis & Prep ---
    # Assumes _parse_composition is defined correctly in the environment
    axes_to_repeat: Dict[str, int] = {}; current_used_axes_lengths = set(used_axes_lengths)
    for ax in output_decomp_flat:
        is_new_axis = ax not in input_axis_names_set and ax != '1'; is_repeat_from_1 = ax != '1' and '1' in input_axis_names_set and ax not in input_axis_names_set
        if is_new_axis or is_repeat_from_1:
            if ax not in axes_lengths: raise EinopsError(f"Unknown axis '{ax}' needs size.")
            size = axes_lengths[ax];
            if not isinstance(size, int) or size <= 0: raise EinopsError(f"Size for '{ax}' invalid.")
            if is_new_axis or (is_repeat_from_1 and axis_sizes.get('1', 0) == 1): axes_to_repeat[ax] = size
            axis_sizes[ax] = size; current_used_axes_lengths.add(ax)
        elif ax == '1': axis_sizes['1'] = 1
    unused_axes_lengths_final = set(axes_lengths.keys()) - current_used_axes_lengths
    if unused_axes_lengths_final: raise EinopsError(f"Provided axes_lengths not used: {unused_axes_lengths_final}")
    output_specs_proc = [spec for spec in output_specs_orig if spec != "..."]
    output_decomp_with_ellipsis: List[str] = list(output_decomp_flat)
    if has_ellipsis:
        num_output_prefix_elem_axes = sum(len(_parse_composition(output_specs_proc[i])) for i in range(right_ellipsis_pos)) # Uses function defined externally
        output_decomp_with_ellipsis.insert(num_output_prefix_elem_axes, '...')

    # --- Permutation (Using Attempt 4 logic - Handles Dropped Axes) ---
    # Assumes _parse_composition is defined correctly in the environment
    source_axes_with_indices: List[Tuple[str, Any]] = []; current_idx = 0; ellipsis_indices: List[int] = []
    if has_ellipsis:
        num_prefix_elems = sum(len(_parse_composition(input_specs_proc[i])) for i in range(left_ellipsis_pos)) # Uses function defined externally
        for i in range(num_prefix_elems): source_axes_with_indices.append((input_decomp_flat[i], current_idx)); current_idx += 1
        ellipsis_indices = list(range(current_idx, current_idx + num_ellipsis_dims))
        if ellipsis_indices: source_axes_with_indices.append(('...', ellipsis_indices))
        current_idx += num_ellipsis_dims
        for i in range(num_prefix_elems, len(input_decomp_flat)): source_axes_with_indices.append((input_decomp_flat[i], current_idx)); current_idx += 1
    else:
        for i in range(len(input_decomp_flat)): source_axes_with_indices.append((input_decomp_flat[i], current_idx)); current_idx += 1
    if current_idx != reshaped_input.ndim: raise EinopsError(f"Internal source axes count error.")
    source_indices_map: Dict[str, List[int]] = {}
    for name, idx_or_list in source_axes_with_indices:
         if name == '...': continue
         if name not in source_indices_map: source_indices_map[name] = []
         source_indices_map[name].append(idx_or_list)

    perm_for_target_axes = []; consumed_source_indices = set()
    for axis_name in output_decomp_with_ellipsis:
        is_existing_in_output = False; origin_axis_name = axis_name
        if axis_name == '...': is_existing_in_output = has_ellipsis and ellipsis_indices
        elif axis_name not in axes_to_repeat: is_existing_in_output = True
        elif axis_name in axes_to_repeat and '1' in source_indices_map: is_existing_in_output = True; origin_axis_name = '1'
        if is_existing_in_output:
            if origin_axis_name == '...':
                perm_for_target_axes.extend(ellipsis_indices); consumed_source_indices.update(ellipsis_indices)
            else:
                source_idx_options = source_indices_map.get(origin_axis_name, []); found_idx = -1
                for idx in source_idx_options:
                    if idx not in consumed_source_indices: found_idx = idx; break
                if found_idx == -1: raise EinopsError(f"Permutation cannot find source for '{axis_name}' (from '{origin_axis_name}').")
                perm_for_target_axes.append(found_idx); consumed_source_indices.add(found_idx)

    all_source_indices = set(range(reshaped_input.ndim))
    dropped_source_indices = sorted(list(all_source_indices - consumed_source_indices))
    final_permutation = perm_for_target_axes + dropped_source_indices

    if len(final_permutation) != reshaped_input.ndim: raise EinopsError(f"Internal Permutation Error: Final map size {len(final_permutation)} != input ndim {reshaped_input.ndim}.")
    if len(set(final_permutation)) != reshaped_input.ndim: raise EinopsError(f"Internal Permutation Error: Duplicate indices in final map: {final_permutation}")

    named_input_indices = set()
    for name, idx_or_list in source_axes_with_indices:
         if name != '1' and name != '...':
              if isinstance(idx_or_list, int): named_input_indices.add(idx_or_list)
    unused_named_indices = named_input_indices.intersection(dropped_source_indices)
    if unused_named_indices:
         first_unused_name = None;
         for name, idx_or_list in source_axes_with_indices:
              if idx_or_list in unused_named_indices: first_unused_name = name; break
         if first_unused_name:
             is_likely_replacement = False
             if first_unused_name not in output_decomp_flat and axes_to_repeat: is_likely_replacement = True
             if not is_likely_replacement: raise EinopsError(f"Input axis '{first_unused_name}' was not used in the output pattern.")

    try:
        if final_permutation == list(range(reshaped_input.ndim)): transposed_tensor = reshaped_input
        else: transposed_tensor = np.transpose(reshaped_input, axes=final_permutation)
    except ValueError as e: raise EinopsError(f"Transpose failed. Shape={reshaped_input.shape}, Permutation={final_permutation}. Error: {e}")

    # --- Repetition --- (Applied after transpose)
    # Assumes _parse_composition is defined correctly in the environment
    tensor_after_repeat = transposed_tensor; axes_inserted_so_far = 0; insertion_points: Dict[str, int] = {}; current_output_dim_idx_post_perm = 0
    for axis_name in output_decomp_with_ellipsis:
         if axis_name == "...": current_output_dim_idx_post_perm += num_ellipsis_dims
         else:
             if axis_name in axes_to_repeat:
                  if axis_name not in insertion_points: insertion_points[axis_name] = current_output_dim_idx_post_perm
             current_output_dim_idx_post_perm += 1
    repeat_targets = []
    for name, size in axes_to_repeat.items():
         if name not in insertion_points: raise EinopsError(f"Internal: No insertion point for '{name}'.")
         repeat_targets.append((name, (insertion_points[name], size)))
    sorted_repeats = sorted(repeat_targets, key=lambda item: item[1][0])
    for axis_name, (target_index, repeat_size) in sorted_repeats:
         actual_insertion_index = target_index + axes_inserted_so_far
         try:
             temp_expanded = np.expand_dims(tensor_after_repeat, axis=actual_insertion_index)
             if repeat_size > 1: tensor_after_repeat = np.repeat(temp_expanded, repeats=repeat_size, axis=actual_insertion_index)
             else: tensor_after_repeat = temp_expanded
             axes_inserted_so_far += 1
         except Exception as e: raise EinopsError(f"Failed insert/repeat '{axis_name}' size {repeat_size} at {actual_insertion_index}. Error: {e}")

    # --- Final Reshape ---
    # Assumes _parse_composition is defined correctly in the environment
    output_shape_elements: List[int] = []
    for spec in output_specs_orig:
        if spec == '...':
             if not has_ellipsis: raise EinopsError("Internal ellipsis error.")
             output_shape_elements.extend(list(ellipsis_shape))
        else:
            components = _parse_composition(spec) # Uses function defined externally
            if not components and spec == '()': output_shape_elements.append(1)
            elif len(components) == 1:
                 axis_name = components[0]; size = axis_sizes.get(axis_name)
                 if size is None:
                     if axis_name == '1': size = 1
                     else: raise EinopsError(f"Internal size error for output axis '{axis_name}'. Sizes: {axis_sizes}")
                 output_shape_elements.append(size)
            else:
                product = 1
                for comp in components:
                    size = axis_sizes.get(comp)
                    if size is None:
                        if comp == '1': size = 1
                        else: raise EinopsError(f"Internal size error for merged component '{comp}'. Sizes: {axis_sizes}")
                    product = np.int64(product) * np.int64(size)
                output_shape_elements.append(int(product))

    final_target_shape_tuple = tuple(output_shape_elements); result = tensor_after_repeat
    if tensor_after_repeat.shape != final_target_shape_tuple:
         expected_elements = np.prod(final_target_shape_tuple, dtype=np.int64); actual_elements = np.prod(tensor_after_repeat.shape, dtype=np.int64)
         if expected_elements != actual_elements: raise EinopsError(f"Cannot perform final reshape. Element count mismatch: {actual_elements} (current) vs {expected_elements} (target). Target shape: {final_target_shape_tuple}. Shape after repeat/transpose: {tensor_after_repeat.shape}. Pattern: '{pattern}'.")
         try: result = np.reshape(tensor_after_repeat, final_target_shape_tuple)
         except ValueError as e: raise EinopsError(f"Failed final reshape from {tensor_after_repeat.shape} to {final_target_shape_tuple}. Error: {e}")

    if result.shape != final_target_shape_tuple: raise EinopsError(f"Internal final shape mismatch.")

    return result

# NO if __name__ == '__main__': block here.
# Testing should be done via the unittest cell (Cell 4)
# which uses the functions defined in Cell 1, 2, and 3.

print("Core rearrange function Defined")

Core rearrange function Defined


In [ ]:
# CELL 4: Unit Tests (Complete & Updated)

import unittest
import numpy as np
# Make sure rearrange is accessible (defined in Cell 3)
# Make sure EinopsError is accessible (defined in Cell 1)

class TestEinopsRearrangeBase(unittest.TestCase):
    """Tests for core rearrangement operations without errors expected."""
    def test_transpose(self):
        x = np.zeros((2, 3, 4)); y = rearrange(x, 'a b c -> c a b'); self.assertEqual(y.shape, (4, 2, 3))
        x = np.zeros((2, 3, 4, 5)); y = rearrange(x, 'a b c d -> a d c b'); self.assertEqual(y.shape, (2, 5, 4, 3))

    def test_split(self):
        x = np.zeros((6, 10)); y = rearrange(x, '(h w) c -> h w c', h=2); self.assertEqual(y.shape, (2, 3, 10))
        x = np.zeros((6, 10)); y = rearrange(x, '(h w) c -> h w c', w=3); self.assertEqual(y.shape, (2, 3, 10))
        x = np.zeros((12, 10)); y = rearrange(x, '(h w) c -> w h c', h=3); self.assertEqual(y.shape, (4, 3, 10))

    def test_merge(self):
        x = np.zeros((2, 3, 4)); y = rearrange(x, 'a b c -> a (b c)'); self.assertEqual(y.shape, (2, 12))
        x = np.zeros((2, 3, 4, 5)); y = rearrange(x, 'a b c d -> (a b) (c d)'); self.assertEqual(y.shape, (6, 20))

    def test_split_merge(self):
        x = np.zeros((6, 10, 3)); y = rearrange(x, '(h w) c d -> h (w c) d', h=2); self.assertEqual(y.shape, (2, 30, 3))
        x = np.zeros((24, 5)); y = rearrange(x, '(a b c) d -> a b (c d)', a=2, b=3); self.assertEqual(y.shape, (2, 3, 20))

    def test_repeat_new_axis(self):
        x = np.zeros((3, 5)); y = rearrange(x, 'h w -> h w c', c=4); self.assertEqual(y.shape, (3, 5, 4))
        x[0, 0] = 1; y = rearrange(x, 'h w -> h w c', c=4); self.assertTrue(np.all(y[0, 0, :] == 1))
        x = np.zeros((3, 5)); y = rearrange(x, 'h w -> c h w', c=4); self.assertEqual(y.shape, (4, 3, 5))

    def test_complex_no_ellipsis(self):
        x = np.zeros((12, 10, 3)); y = rearrange(x, '(h w) c d -> w (c d) h', h=3); self.assertEqual(y.shape, (4, 30, 3))
        x = np.zeros((6, 20)); y = rearrange(x, '(h w) (c d) -> h w c d', h=2, c=4); self.assertEqual(y.shape, (2, 3, 4, 5))
        x = np.zeros((2, 3, 4, 5)); y = rearrange(x, 'a b c d -> (a b c) d'); self.assertEqual(y.shape, (24, 5))

    def test_repeat_from_one(self):
        x = np.zeros((3, 1, 5)); y = rearrange(x, 'a 1 c -> a b c', b=4); self.assertEqual(y.shape, (3, 4, 5))
        x[0, 0, 0] = 1; y = rearrange(x, 'a 1 c -> a b c', b=4); self.assertTrue(np.all(y[0, :, 0] == 1))

    def test_ellipsis_simple(self):
        x = np.zeros((10, 2, 3, 4)); y = rearrange(x, '... b c d -> ... d c b'); self.assertEqual(y.shape, (10, 4, 3, 2))
        x = np.zeros((10, 11, 6, 5)); y = rearrange(x, '... (h w) c -> ... h w c', h=2); self.assertEqual(y.shape, (10, 11, 2, 3, 5))
        x = np.zeros((10, 11, 2, 3, 5)); y = rearrange(x, '... h w c -> ... (h w) c'); self.assertEqual(y.shape, (10, 11, 6, 5))
        x = np.zeros((10, 3, 1, 5)); y = rearrange(x, '... a 1 c -> ... a b c', b=4); self.assertEqual(y.shape, (10, 3, 4, 5))
        x = np.zeros((2, 3, 4, 10, 11)); y = rearrange(x, 'a b c ... -> c a b ...'); self.assertEqual(y.shape, (4, 2, 3, 10, 11))

class TestEinopsRearrangeAdvanced(unittest.TestCase):
    """Tests for more complex operations and combinations."""
    def test_ellipsis_middle(self):
        x = np.zeros((2, 3, 4, 5, 6)); y = rearrange(x, 'a b ... c -> c b ... a'); self.assertEqual(y.shape, (6, 3, 4, 5, 2))
        x = np.zeros((2, 3, 4, 5, 6)); y = rearrange(x, 'a ... c -> c ... a'); self.assertEqual(y.shape, (6, 3, 4, 5, 2))

    def test_nested_parentheses(self):
        x = np.zeros((24, 5)); y = rearrange(x, '((a b) c) d -> a b c d', a=2, b=3); self.assertEqual(y.shape, (2, 3, 4, 5))
        x = np.zeros((2, 3, 4, 5)); y = rearrange(x, 'a b c d -> (a (b c)) d'); self.assertEqual(y.shape, (24, 5))
        x = np.zeros((6, 20)); y = rearrange(x, '(h (w1 w2)) (c d) -> h w1 w2 c d', h=2, w1=3, d=5); self.assertEqual(y.shape, (2, 3, 1, 4, 5)) # Note: w2 becomes 1 if 6 = 2 * 3 * w2
        x = np.zeros((2,3,4,5,6)); y = rearrange(x, 'a b c d e -> (a (b (c d))) e'); self.assertEqual(y.shape, (120, 6))

    def test_ellipsis_split_merge(self):
        x = np.zeros((6, 5, 10, 11)); y = rearrange(x, '(h w) c ... -> h w c ...', h=2); self.assertEqual(y.shape, (2, 3, 5, 10, 11))
        x = np.zeros((10, 11, 6, 5)); y = rearrange(x, '... t (h w) c -> ... t h w c', h=2); self.assertEqual(y.shape, (10, 11, 2, 3, 5))
        x = np.zeros((10, 11, 2, 3, 5)); y = rearrange(x, '... t h w c -> ... t (h w) c'); self.assertEqual(y.shape, (10, 11, 6, 5))
        x = np.zeros((10, 2, 3, 4, 5)); y = rearrange(x, 'b h w ... -> b (h w) ...'); self.assertEqual(y.shape, (10, 6, 4, 5))
        x = np.zeros((10, 2, 3, 4, 5)); y = rearrange(x, 'b pre ... post d -> b (pre post) ... d'); self.assertEqual(y.shape, (10, 8, 3, 5)) # Fails if ellipsis is empty, ensure tensor has enough dims
        x = np.zeros((10, 6, 3, 8, 5)); y = rearrange(x, 'b (pre1 pre2) ... (post1 post2) d -> b pre1 pre2 ... post1 post2 d', pre1=2, post2=4); self.assertEqual(y.shape, (10, 2, 3, 3, 2, 4, 5))

    def test_repeat_non_one_dimension(self): # Test adjusted for V5/V6 logic
        x = np.zeros((2, 3, 4)); y = rearrange(x, 'a b c -> a b c d', d=5); self.assertEqual(y.shape, (2, 3, 4, 5))
        x[0,0,0]=1; y = rearrange(x, 'a b c -> a b c d', d=5); self.assertTrue(np.all(y[0,0,0,:] == 1))

        # Test replacing an axis with different size - should fail at final reshape/element check
        x = np.zeros((2, 6, 4))
        # --- ADJUSTED ASSERTION ---
        # Expect the element count mismatch error before final reshape
        with self.assertRaisesRegex(EinopsError, r"Element count mismatch", msg="Replacing dim 6 with size 5 should fail due to element count"):
            rearrange(x, 'a b c -> a d c', d=5)

        # Test replacing an axis with the same size - should pass
        x = np.zeros((2, 6, 4))
        y = rearrange(x, 'a b c -> a d c', d=6)
        self.assertEqual(y.shape, (2, 6, 4))


class TestEinopsErrorHandling(unittest.TestCase):
    """Tests for various error conditions."""

    def test_error_invalid_pattern(self):
        x = np.zeros((2,3))
        with self.assertRaisesRegex(EinopsError, "Pattern must contain '->'"):
            rearrange(x, 'a b')
        with self.assertRaisesRegex(EinopsError, "Ellipsis .* both sides or neither"):
            rearrange(x, '... a b -> a b')
        # --- ADJUSTED ASSERTION ---
        with self.assertRaisesRegex(EinopsError, r"Invalid pattern.*Ellipsis \(\.\.\.\) max once on left"):
             rearrange(x, 'a ... b ... -> a b')
        with self.assertRaisesRegex(EinopsError, "Invalid identifier found: 'a-b'"):
            rearrange(x, 'a-b -> a b')
        with self.assertRaisesRegex(EinopsError, "Mismatched parentheses"):
            rearrange(x, '(a b -> a b')
        with self.assertRaisesRegex(EinopsError, "Mismatched parentheses"):
            rearrange(x, 'a (b (c d) -> a b c d')
        # --- ADJUSTED ASSERTION ---
        with self.assertRaisesRegex(EinopsError, r"Invalid pattern.*Duplicate axes on left: \['a'\]"):
            rearrange(x, 'a a -> a')

    def test_error_dim_mismatch(self):
        x = np.zeros((2, 3))
        # --- ADJUSTED ASSERTION ---
        with self.assertRaisesRegex(EinopsError, r"Tensor ndim 2 != pattern specs 3"):
             rearrange(x, 'a b c -> b a c')
        with self.assertRaisesRegex(EinopsError, r"Tensor ndim 2 != pattern specs 1"):
             rearrange(x, 'a -> a')
        x = np.zeros((5, 2, 3))
        # This case should now pass
        y = rearrange(x, 'a ... -> ... a')
        self.assertEqual(y.shape, (2, 3, 5))

    def test_error_split_inconsistent(self):
        x = np.zeros((7, 10))
        with self.assertRaisesRegex(EinopsError, "not divisible"):
            rearrange(x, '(h w) c -> h w c', h=2)
        x = np.zeros((6, 10))
        with self.assertRaisesRegex(EinopsError, r"Cannot infer sizes for \['h', 'w'\]"): # Use pattern from your original tests if different
            rearrange(x, '(h w) c -> h w c')
        x = np.zeros((6, 10))
        # --- ADJUSTED ASSERTION ---
        with self.assertRaisesRegex(EinopsError, r"Product 8 != dim 6 for '\(h w\)'\."):
            rearrange(x, '(h w) c -> h w c', h=2, w=4)
        # Duplicate axes check is in test_error_invalid_pattern

    def test_error_axis_length(self):
        x = np.zeros((3, 5))
        # --- ADJUSTED ASSERTION ---
        with self.assertRaisesRegex(EinopsError, "Unknown axis 'c' needs size"):
             rearrange(x, 'h w -> h w c')
        x = np.zeros((3,5))
        with self.assertRaisesRegex(EinopsError, "Provided axes_lengths not used: {'d'}"):
            rearrange(x, 'h w -> h w c', c=4, d=5)
        x = np.zeros((6, 10))
        # Should pass
        rearrange(x, '(h w) c -> h w c', h=2, w=3)
        with self.assertRaisesRegex(EinopsError, "Provided axes_lengths not used: {'z'}"):
             rearrange(x, '(h w) c -> h w c', h=2, z=99)
        x = np.zeros((6, 10))
        # --- ADJUSTED ASSERTION ---
        with self.assertRaisesRegex(EinopsError, r"Size for 'h' invalid"):
            rearrange(x, '(h w) c -> h w c', h=-2)

    def test_error_input_axis_unused(self):
        x = np.zeros((2, 3, 4))
        # --- ADJUSTED ASSERTION ---
        # Now correctly expects the unused axis error
        with self.assertRaisesRegex(EinopsError, "Input axis 'c' was not used"):
            rearrange(x, 'a b c -> a b')
        x = np.zeros((6, 4))
        # --- ADJUSTED ASSERTION ---
        # Now correctly expects the unused axis error
        with self.assertRaisesRegex(EinopsError, "Input axis 'b' was not used"):
             rearrange(x, '(a b) c -> a c', a=2)

    def test_error_empty_composition(self):
        x = np.zeros((2, 3))
        # --- ADJUSTED ASSERTION ---
        with self.assertRaisesRegex(EinopsError,"Spec '\\(\\)' requires 1, got 2"):
            rearrange(x, '() b -> b')
        x = np.zeros((1, 3))
        # Should pass
        y = rearrange(x, '() b -> b')
        self.assertEqual(y.shape, (3,))

    def test_error_unknown_axis_in_output(self):
        x = np.zeros((2, 3))
        # --- ADJUSTED ASSERTION ---
        with self.assertRaisesRegex(EinopsError, "Unknown axis 'd' needs size"):
             rearrange(x, 'a b -> (a d)')

# --- Test Runner ---
def run_tests():
   suite = unittest.TestSuite()
   loader = unittest.TestLoader()
   # Load tests from all classes
   suite.addTest(loader.loadTestsFromTestCase(TestEinopsRearrangeBase))
   suite.addTest(loader.loadTestsFromTestCase(TestEinopsRearrangeAdvanced))
   suite.addTest(loader.loadTestsFromTestCase(TestEinopsErrorHandling))
   print("Running Unit Tests...\n")
   # Use default runner which works well in notebooks
   runner = unittest.TextTestRunner(verbosity=2)
   result = runner.run(suite)
   print("\nTest Run Complete.")
   # Check results
   if result.wasSuccessful():
       print("\n---------------------")
       print("🎉 All tests passed! 🎉")
       print("---------------------")
   else:
       print("\n---------------------")
       print(f"⚠️ FAILURES/ERRORS occurred (Failures={len(result.failures)}, Errors={len(result.errors)}) ⚠️")
       print("---------------------")

# Execute the tests
run_tests()

test_complex_no_ellipsis (__main__.TestEinopsRearrangeBase.test_complex_no_ellipsis) ... ok
test_ellipsis_simple (__main__.TestEinopsRearrangeBase.test_ellipsis_simple) ... ok
test_merge (__main__.TestEinopsRearrangeBase.test_merge) ... ok
test_repeat_from_one (__main__.TestEinopsRearrangeBase.test_repeat_from_one) ... ok
test_repeat_new_axis (__main__.TestEinopsRearrangeBase.test_repeat_new_axis) ... ok
test_split (__main__.TestEinopsRearrangeBase.test_split) ... ok
test_split_merge (__main__.TestEinopsRearrangeBase.test_split_merge) ... ok
test_transpose (__main__.TestEinopsRearrangeBase.test_transpose) ... ok
test_ellipsis_middle (__main__.TestEinopsRearrangeAdvanced.test_ellipsis_middle) ... ok
test_ellipsis_split_merge (__main__.TestEinopsRearrangeAdvanced.test_ellipsis_split_merge) ... ok
test_nested_parentheses (__main__.TestEinopsRearrangeAdvanced.test_nested_parentheses) ... ok
test_repeat_non_one_dimension (__main__.TestEinopsRearrangeAdvanced.test_repeat_non_one_dimension) .

Running Unit Tests...


Test Run Complete.

---------------------
⚠️ FAILURES/ERRORS occurred (Failures=0, Errors=1) ⚠️
---------------------
